In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear la sesión de Spark con los JARs necesarios
spark = SparkSession.builder \
    .appName("KafkaConnectionTest") \
    .config("spark.jars", "/usr/local/spark/jars/*") \
    .getOrCreate()

spark.sparkContext.setLogLevel("DEBUG")


# Intenta leer el stream de Kafka
try:
    df_raw = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:9092") \
        .option("subscribe", "invernadero") \
        .option("startingOffsets", "earliest") \
        .load()

    # Muestra el esquema para verificar si Spark se conectó exitosamente a Kafka
    df_raw.printSchema()

    # Selecciona el valor de los mensajes y conviértelo a cadena
    df_string = df_raw.selectExpr("CAST(value AS STRING) as message")

    # Inicia la consulta para mostrar algunos datos del stream
    query = df_string.writeStream \
        .format("console") \
        .option("truncate", "false") \
        .start()

    # Espera un momento para ver los datos (ajusta el tiempo según sea necesario)
    query.awaitTermination(10)  # espera 10 segundos antes de detener la consulta
    query.stop()

except Exception as e:
    print("Error al conectarse a Kafka:", e)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

